In [1]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get the variables from the environment
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

In [2]:
# Connect to PostgreSQL database
try:
    connection = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password,
        port=db_port
    )
    
    # Create a cursor object
    cursor = connection.cursor()

    # Execute SQL queries to get data from both tables
    query_campaign = "SELECT * FROM campaign"  # Query to fetch data from campaign table
    query_contacts = "SELECT * FROM contacts"  # Query to fetch data from contacts table

    # Fetch campaign data
    cursor.execute(query_campaign)
    data_campaign = cursor.fetchall()
    col_names_campaign = [desc[0] for desc in cursor.description]

    # Fetch contacts data
    cursor.execute(query_contacts)
    data_contacts = cursor.fetchall()
    col_names_contacts = [desc[0] for desc in cursor.description]

    # Create Pandas DataFrames from the fetched data
    df_campaign = pd.DataFrame(data_campaign, columns=col_names_campaign)
    df_contacts = pd.DataFrame(data_contacts, columns=col_names_contacts)

    # Merge the two DataFrames on 'contact_id'
    df_merged = pd.merge(df_campaign, df_contacts, on='contact_id', how='inner')  # 'inner' join can be replaced by 'left' or 'right' if needed

    # Display the merged DataFrame
    print(df_merged)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


     cf_id  contact_id                  company_name  \
0      147        4661    Baldwin, Riley and Jackson   
1     1621        3765                      Odom Inc   
2     1812        4187    Melton, Robinson and Fritz   
3     2156        4941   Mcdonald, Gonzalez and Ross   
4     1365        2199                 Larson-Little   
..     ...         ...                           ...   
995   2986        3684              Manning-Hamilton   
996   2031        5784                    Butler LLC   
997   1627        1498                      Ball LLC   
998   2175        6073   Taylor, Santiago and Flores   
999   1788        4939  Hernandez, Norton and Kelley   

                                       description      goal   pledged  \
0             Pre-emptive tertiary standardization     100.0       0.0   
1                 Managed bottom-line architecture    1400.0   14560.0   
2     Function-based leadingedge pricing structure  108400.0  142523.0   
3    Vision-oriented fresh-thin

In [4]:
# Example: Show the first 5 rows
df_merged.head()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id,first_name,last_name,email
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13,2021-03-01,cat1,subcat1,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25,2021-05-25,cat2,subcat2,Mariana,Ellis,mariana.ellis@rossi.org
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,2020-12-17,2021-12-30,cat3,subcat3,Sofie,Woods,sofie.woods@riviere.com
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,2021-10-21,2022-01-17,cat2,subcat2,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,2020-12-21,2021-08-23,cat4,subcat4,Samuel,Sorgatz,samuel.sorgatz@gmail.com


In [5]:
# Example: Get summary statistics
df_merged.describe()

,cf_id,contact_id,goal,pledged,backers_count
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1585.743000,3751.434000,43983.100000,42748.055000,727.005000
std,926.944534,1431.743284,58962.425595,57415.481551,1137.723135
min,23.000000,1235.000000,100.000000,0.000000,0.000000
25%,758.750000,2502.750000,4200.000000,5911.250000,86.000000
50%,1571.500000,3706.500000,8300.000000,11950.000000,184.500000
75%,2402.250000,4994.250000,80625.000000,66294.250000,923.750000
max,3209.000000,6234.000000,199200.000000,199110.000000,7295.000000
